In [7]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

import gi
gi.require_version("Gtk", "3.0")
gi.require_version("Gdk", "3.0")
from gi.repository import Gdk, Gtk
gi.require_foreign("cairo")

plt.rcParams["figure.figsize"] = (20, 10)

Przerób przykład z rysowaniem myszką odcinka tak, aby obsługiwał wiele odcinków (tzn. trzecie kliknięcie myszki powinno wyznaczać początkowy punkt nowego odcinka, a nie wymazywać stary). Potrzebna będzie jakaś struktura danych potrafiąca przechowywać wiele liczb — najprościej będzie użyć listy (patrz dokumentacja Pythona).

In [8]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Line:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
class Okno(Gtk.Window):
    def __init__(self):
        super(Okno, self).__init__()
        
        self.lines = [Line(None, None)]

        self.set_title("Okno GTK")
        self.set_default_size(400, 300)
        self.connect("destroy", Gtk.main_quit)
        
        da = Gtk.DrawingArea()
        da.set_size_request(200, 200)
        
        da.connect("draw", self.on_draw)
        da.set_events(Gdk.EventMask.BUTTON_PRESS_MASK)
        da.connect("button-press-event", self.on_button_press)
        
        self.add(da)
        self.show_all()

    def on_draw(self, widget, ctx):
        for line in self.lines:
            if line.p1 is not None:
                ctx.arc(line.p1.x, line.p1.y, 8, 0.0, 6.283)    # ciut mniej niż 2π
                ctx.close_path()
                ctx.stroke()

            if line.p2 is not None:
                ctx.move_to(line.p1.x, line.p1.y)
                ctx.line_to(line.p2.x, line.p2.y)
                ctx.stroke()

    def on_button_press(self, widget, event):
        if event.button != 1:
            return False
        
        last_line = self.lines[-1]

        if last_line.p1 is None:
            last_line.p1 = Point(event.x, event.y)
        elif last_line.p2 is None:
            last_line.p2 = Point(event.x, event.y)
            self.lines.append(Line(None, None))

        widget.queue_draw()

        return True

In [9]:
o = Okno()
Gtk.main()